# Algo de novo em Python 3

Veja estes exemplos em Python 2:

```python
>>> L = [0, 1, 2]
>>> zip(L, 'ABC')
[(0, 'A'), (1, 'B'), (2, 'C')]
>>> map(lambda x: x*10, L)
[0, 10, 20]
>>> filter(None, L)
[1, 2]
```

Agora os mesmos comandos em Python 3:

In [1]:
L = [0, 1, 2]
zip(L, 'ABC')

In [2]:
map(lambda x: x*10, L)

In [3]:
filter(None, L)

É como diz o Hélio Meira Lins: quase nada em Python 3 devolve uma lista.

Para construir listas, use o construtor `list`!

In [4]:
list(zip(L, 'ABC'))

[(0, 'A'), (1, 'B'), (2, 'C')]

In [5]:
list(map(lambda x: x*10, L))

[0, 10, 20]

In [6]:
list(filter(None, L))

[1, 2]

Porém esses últimos exemplos eu faria assim:

In [7]:
[x*10 for x in L]

[0, 10, 20]

In [8]:
[x for x in L if x]

[1, 2]

E a versão *lazy* (preguiçosa), eu faria assim:

In [9]:
(x*10 for x in L)

<generator object <genexpr> at 0x105360af0>

A novidade é uso o ainda mais extensivo de geradores em Python 3.

# Como funciona um objeto gerador

Objetos geradores são iteradores: oferecem um método `__next__` que, quando acionado pela função embutida `next`, produzem o próximo item de uma série. A série pode ser limitada ou ilimitada.

In [10]:
z = zip(L, 'ABC')
z

In [11]:
next(z)

(0, 'A')

In [12]:
next(z)

(1, 'B')

In [13]:
next(z)

(2, 'C')

## Quando esgotado, um gerador levanta `StopIteration`

In [14]:
# next(z)

Não há como reiniciar um gerador. Se for preciso percorrer duas vezes, é preciso jogá-lo fora e começar outro:

In [15]:
z = zip(L, 'ABC')
next(z)

(0, 'A')

# Trem iterável

In [19]:
class Trem(object):

    def __init__(self, vagoes):
        self.vagoes = vagoes

    def __iter__(self):
        return IteradorTrem(self.vagoes)

class IteradorTrem(object):

    def __init__(self, vagoes):
        self.atual = 0
        self.ultimo_vagao = vagoes - 1

    def __next__(self):
        if self.atual <= self.ultimo_vagao:
            self.atual += 1
            return 'vagão #%s' % (self.atual)
        else:
            raise StopIteration()

In [24]:
t = Trem(3)
for vagão in t:
    print(vagão)

vagão #1
vagão #2
vagão #3


In [25]:
it = iter(t)
it

In [26]:
next(it)

'vagão #1'

In [27]:
next(it)

'vagão #2'

In [28]:
next(it)

'vagão #3'

In [29]:
next(it)

StopIteration: 

In [30]:
class Trem(object):

    def __init__(self, vagoes):
        self.vagoes = vagoes

    def __iter__(self):
        for i in range(self.vagoes):
            yield 'vagão #%s' % (i+1)

In [32]:
t = Trem(3)
it = iter(t)
it

<generator object __iter__ at 0x10539f4c0>

In [33]:
next(it), next(it), next(it)

('vagão #1', 'vagão #2', 'vagão #3')

In [34]:
next(it)

StopIteration: 